《[利用Python进行数据分析](https://book.douban.com/subject/25779298/)》读书笔记。
 
 [第10章](/2017/07/20/python_data_analysis10.html)  第5节：时期及其算数运算

所有用到的数据可以从[作者的 github](https://github.com/wesm/pydata-book)下载。


In [1]:
%pylab inline
import pandas as pd
from datetime import datetime
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib


时期（period）,表示时间区间，比如几日、几月、几年等。

Period类所表示的就是这种数据类型，其构造函数需要用到一个字符串或整数，以及频率。

In [2]:
p = pd.Period(2007, freq='A-DEC')
p

Period('2007', 'A-DEC')

In [3]:
# 位移
p + 5

Period('2012', 'A-DEC')

In [4]:
# 相同频率的Period可以进行加减,不同频率是不能加减的
pd.Period('2014', freq='A-DEC') - p

7

In [5]:
# period_range函数，可用于创建规则的时期范围

rng = pd.period_range('1/1/2000', '6/30/2000', freq='M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

In [6]:
# 将 PeriodIndex 作为索引
Series(np.random.randn(6), index=rng)

2000-01   -1.057696
2000-02    0.409239
2000-03    0.500589
2000-04    0.842015
2000-05   -1.538833
2000-06    0.735769
Freq: M, dtype: float64

In [7]:
# 直接使用一组字符串构建
values = ['2001Q3', '2002Q2', '2003Q1']
index = pd.PeriodIndex(values, freq='Q-DEC')
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

## 时期的频率转换

Period和PeriodIndex对象都可以通过其asfreq方法转换为别的频率。

In [8]:
# 将年度时期转换为月度时期
p = pd.Period('2007', freq='A-DEC')
p.asfreq('M', how='start')

Period('2007-01', 'M')

In [9]:
p.asfreq('M', how='end')

Period('2007-12', 'M')

In [10]:
p = pd.Period('2007', freq='A-JUN')
p.asfreq('M', 'start')

Period('2006-07', 'M')

In [12]:
# 高频率时期转换为低频率时期
p = pd.Period('Aug-2007', 'M')

# 注意， 2007-08,属于周期2008年
p.asfreq('A-JUN')

Period('2008', 'A-JUN')

In [14]:
# PeriodIndex 或 TimeSeries 的频率转换
rng = pd.period_range('2006', '2009', freq='A-DEC')
ts = Series(np.random.randn(len(rng)), index=rng)
ts

2006   -1.195180
2007    0.424639
2008    0.157479
2009    0.774099
Freq: A-DEC, dtype: float64

In [15]:
ts.asfreq('M', how='start')

2006-01   -1.195180
2007-01    0.424639
2008-01    0.157479
2009-01    0.774099
Freq: M, dtype: float64

In [16]:
ts.asfreq('B', how='end')

2006-12-29   -1.195180
2007-12-31    0.424639
2008-12-31    0.157479
2009-12-31    0.774099
Freq: B, dtype: float64

## 按季度计算的时期频率

季度型数据在会计、金融等领域中很常见。许多季度型数据都会涉及“财年末”的概念，通常是一年12个月中某月的最后一个日历日或工作日。就这一点来说，“2012Q4”根据财年末的会有不同含义。pandas支持12种可能的季度频率，即Q-JAN、Q-DEC。

In [17]:
p = pd.Period('2012Q4', freq='Q-JAN')
p

Period('2012Q4', 'Q-JAN')

In [18]:
p.asfreq('D', 'start')

Period('2011-11-01', 'D')

In [19]:
p.asfreq('D', 'end')

Period('2012-01-31', 'D')

In [21]:
p4pm = (p.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
p4pm

Period('2012-01-30 16:00', 'T')

In [22]:
p4pm.to_timestamp()

Timestamp('2012-01-30 16:00:00')

In [23]:
# period_range还可以用于生产季度型范围，季度型范围的算数运算也跟上面是一样的
rng = pd.period_range('2011Q3', '2012Q4', freq='Q-JAN')
ts = Series(np.arange(len(rng)), index=rng)
ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int32

In [24]:
new_rng = (rng.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
ts.index = new_rng.to_timestamp()
ts

2010-10-28 16:00:00    0
2011-01-28 16:00:00    1
2011-04-28 16:00:00    2
2011-07-28 16:00:00    3
2011-10-28 16:00:00    4
2012-01-30 16:00:00    5
dtype: int32

## Timestamp与Period的互相转换

通过to_period方法，可以将由时间戳索引的Series和DataFrame对象转换为以时期为索引的对象

In [25]:
rng = pd.date_range('1/1/2000', periods=3, freq='M')
ts = Series(randn(3), index=rng)
pts = ts.to_period()
ts

2000-01-31   -0.890889
2000-02-29    0.863907
2000-03-31   -0.734717
Freq: M, dtype: float64

In [26]:
pts

2000-01   -0.890889
2000-02    0.863907
2000-03   -0.734717
Freq: M, dtype: float64

In [30]:
# 由于时期指的是非重叠时间区间，因此对于给定的频率，一个时间戳只能属于一个时期。
# 新PeriodIndex的频率默认是从时间戳推断而来的，当然可以自己指定频率，结果中允许存在重复时期

rng = pd.date_range('1/29/2000', periods=6, freq='D')
ts2 = Series(randn(6), index=rng)
ts2.to_period('M')

2000-01    0.974506
2000-01   -0.175840
2000-01    0.051886
2000-02   -0.543674
2000-02    1.710834
2000-02    0.343342
Freq: M, dtype: float64

In [28]:
pts = ts.to_period()
pts

2000-01   -0.890889
2000-02    0.863907
2000-03   -0.734717
Freq: M, dtype: float64

In [31]:
# 转换为时间戳
pts.to_timestamp(how='end')

2000-01-31   -0.890889
2000-02-29    0.863907
2000-03-31   -0.734717
Freq: M, dtype: float64

## 通过数组创建PeriodIndex

固定频率的数据集通常会将时间信息分开存放在多个列中。例如下面的这个宏观经济数据集中，年度和季度就分别存放在不同的列中。

In [33]:
data = pd.read_csv('data/ch08/macrodata.csv')
data.year.head()

0    1959.0
1    1959.0
2    1959.0
3    1959.0
4    1960.0
Name: year, dtype: float64

In [34]:
data.quarter.head()

0    1.0
1    2.0
2    3.0
3    4.0
4    1.0
Name: quarter, dtype: float64

In [35]:
# 使用 year, quarter这两个数组，以及 一个频率 Q-DEC, 构建一个 PeriodIndex
index = pd.PeriodIndex(year=data.year, quarter=data.quarter, freq='Q-DEC')
index

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203, freq='Q-DEC')

In [38]:
# 将 PeriodIndex 作为 data 的索引
data.index = index
data.infl.head()

1959Q1    0.00
1959Q2    2.34
1959Q3    2.74
1959Q4    0.27
1960Q1    2.31
Freq: Q-DEC, Name: infl, dtype: float64